In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame,Series
import seaborn as sns

C:\Users\aaa\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\aaa\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\aaa\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


## 6.3.1 transform方法

### 首先对小费数据集新建一列用于存放男性和女性小费的平均值。常用的方法是，先聚合运算，然后再将其合并

In [3]:
tips = sns.load_dataset('tips')

In [4]:
df = DataFrame(tips.groupby('sex')['tip'].mean())
df

,tip
sex,
Male,3.089618
Female,2.833448


In [6]:
new_tips = pd.merge(tips,df,left_on='sex',right_index=True)
new_tips.head()

,total_bill,tip_x,sex,smoker,day,time,size,tip_y
0,16.99,1.01,Female,No,Sun,Dinner,2,2.833448
4,24.59,3.61,Female,No,Sun,Dinner,4,2.833448
11,35.26,5.00,Female,No,Sun,Dinner,4,2.833448
14,14.83,3.02,Female,No,Sun,Dinner,2,2.833448
16,10.33,1.67,Female,No,Sun,Dinner,3,2.833448


### 上面的方法虽然也能实现，但过于烦琐，不灵活。通过transform方法可以使运算分布到每一行

In [7]:
tips.groupby('sex')['tip'].transform('mean')

0      2.833448
1      3.089618
2      3.089618
3      3.089618
4      2.833448
         ...   
239    3.089618
240    2.833448
241    3.089618
242    3.089618
243    2.833448
Name: tip, Length: 244, dtype: float64

## 6.3.2 apply方法

### apply方法的功能更加强大，例如可以计算根据性别分组后小费金额排在前5名的DataFrame数据

In [8]:
def top(x,n=5):
    return x.sort_values(by='tip',ascending=False)[-n:]

In [9]:
tips.groupby('sex').apply(top)

total_bill   tip     sex smoker  day    time  size
sex                                                           
Male   43         9.68  1.32    Male     No  Sun  Dinner     2
       235       10.07  1.25    Male     No  Sat  Dinner     2
       75        10.51  1.25    Male     No  Sat  Dinner     2
       237       32.83  1.17    Male    Yes  Sat  Dinner     2
       236       12.60  1.00    Male    Yes  Sat  Dinner     2
Female 215       12.90  1.10  Female    Yes  Sat  Dinner     2
       0         16.99  1.01  Female     No  Sun  Dinner     2
       111        7.25  1.00  Female     No  Sat  Dinner     1
       67         3.07  1.00  Female    Yes  Sat  Dinner     1
       92         5.75  1.00  Female    Yes  Fri  Dinner     2

### 如果希望返回的结果不以分组键为索引，通过group_keys=False可以完成

In [10]:
tips.groupby('sex',group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size
43,9.68,1.32,Male,No,Sun,Dinner,2
235,10.07,1.25,Male,No,Sat,Dinner,2
75,10.51,1.25,Male,No,Sat,Dinner,2
237,32.83,1.17,Male,Yes,Sat,Dinner,2
236,12.60,1.00,Male,Yes,Sat,Dinner,2
215,12.90,1.10,Female,Yes,Sat,Dinner,2
0,16.99,1.01,Female,No,Sun,Dinner,2
111,7.25,1.00,Female,No,Sat,Dinner,1
67,3.07,1.00,Female,Yes,Sat,Dinner,1
92,5.75,1.00,Female,Yes,Fri,Dinner,2


### 例如，前面对缺失数据的处理可通过数值填充来完成

In [12]:
data ={
    'name':['张三','李四','peter','王五','小明','小红'],
    'sex':['female','female','male','male','male','female'],
    'math':[67,73,np.nan,82,90,np.nan]
}
df = DataFrame(data)
df['math'] = df['math']
df

,name,sex,math
0,张三,female,67.0
1,李四,female,73.0
2,peter,male,NaN
3,王五,male,82.0
4,小明,male,90.0
5,小红,female,NaN


In [13]:
df.fillna(df['math'].mean())

,name,sex,math
0,张三,female,67.0
1,李四,female,73.0
2,peter,male,78.0
3,王五,male,82.0
4,小明,male,90.0
5,小红,female,78.0


### 可以这样假设：男生和女生的数学成绩还是有区别的，希望通过分组后，再进行插值

In [14]:
f = lambda x :x.fillna(x.mean())
df.groupby('sex').apply(f)

name     sex  math
sex                          
female 0     张三  female  67.0
       1     李四  female  73.0
       5     小红  female  70.0
male   2  peter    male  86.0
       3     王五    male  82.0
       4     小明    male  90.0